In [7]:
import sys
print(sys.executable)

from lulab.utils.paths import repo_root
print("Repo root:", repo_root())

from lulab.utils.topic import get_topic_dir, topic_paths

TOPIC_DIR = get_topic_dir()
PATHS = topic_paths(TOPIC_DIR)

print(TOPIC_DIR)
print(PATHS["figures"])
print(PATHS["scripts"])
print(PATHS["tex"])
print(PATHS["tables"])
print(PATHS["meta"])


/Users/mloktionov/anaconda3/envs/astro-ai/bin/python
Repo root: /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab
/Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius
/Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/figures
/Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/scripts
/Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/tex
/Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/tables
/Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/meta


In [9]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
# from lulab.viz.export import save_figure

# topic paths
NOTEBOOK_DIR = Path.cwd()
TOPIC_DIR = NOTEBOOK_DIR.parent   # notebooks -> TOPIC_DIR
OUT = TOPIC_DIR / "figures"
OUT.mkdir(parents=True, exist_ok=True)

# dummy data
x = np.linspace(0, 10, 200)
y = np.sin(x)

fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_title("Dummy plot (pipeline check)")
ax.set_xlabel("x")
ax.set_ylabel("sin(x)")

# save
outpath = OUT / "FIG_000_pipeline_check.png"
fig.savefig(outpath, dpi=150, bbox_inches="tight")
# save_figure(fig, PATHS["figures"] / "FIG_000_pipeline_check", formats=("png",))
print("Saved:", outpath)
plt.close(fig)

Saved: /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/figures/FIG_000_pipeline_check.png


In [1]:
from pathlib import Path
import re

log_dir = Path("build/logs")
failed = ["ACAP_001_EN", "ACAP_001_RU", "AZ_001_EN", "AZ_001_RU", "AZ_002_EN"]

def root_cause(stderr_text: str) -> str:
    # ищем последнюю строку вида "XxxError: ...."
    lines = stderr_text.strip().splitlines()
    pat = re.compile(r"^(\w*(?:Error|Exception)):\s*(.*)$")
    for line in reversed(lines):
        m = pat.match(line.strip())
        if m:
            return f"{m.group(1)}: {m.group(2)}"
    # fallback: если нет стандартной строки
    return (lines[-1].strip() if lines else "No stderr captured")

for name in failed:
    p = log_dir / f"{name}.stderr.log"
    if not p.exists():
        print(f"{name}: stderr log missing: {p}")
        continue
    txt = p.read_text(errors="replace")
    print(f"{name} -> {root_cause(txt)}")

ACAP_001_EN: stderr log missing: build/logs/ACAP_001_EN.stderr.log
ACAP_001_RU: stderr log missing: build/logs/ACAP_001_RU.stderr.log
AZ_001_EN: stderr log missing: build/logs/AZ_001_EN.stderr.log
AZ_001_RU: stderr log missing: build/logs/AZ_001_RU.stderr.log
AZ_002_EN: stderr log missing: build/logs/AZ_002_EN.stderr.log
